In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=4f00699fb6f8a4d318c78a31a9e205cb125361072d2add9cad60b0d691eca5d0
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

In [5]:
# create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2023-01-07 10:38:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  17.9MB/s    in 0.7s    

2023-01-07 10:38:12 (17.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://c033-34-125-30-224.ngrok.io


In [7]:
data = [("James", "Sales", 3000), 
    ("Michael", "Sales", 4600), 
    ("Robert", "Sales", 4100), 
    ("Maria", "Finance", 3000), 
    ("James", "Sales", 3000), 
    ("Scott", "Finance", 3300), 
    ("Jen", "Finance", 3900), 
    ("Jeff", "Marketing", 3000), 
    ("Kumar", "Marketing", 2000), 
    ("Saif", "Sales", 4100) 
  ]

In [8]:
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = data, schema = columns)

In [9]:
df

DataFrame[employee_name: string, department: string, salary: bigint]

In [11]:
df.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [12]:
distinctDF = df.distinct()

In [13]:
distinctDF.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|      Michael|     Sales|  4600|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|          Jen|   Finance|  3900|
|        Scott|   Finance|  3300|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [14]:
distinctDF.count()

9

In [15]:
dropDisDF = df.dropDuplicates(["department","salary"])
print("Distinct count of department & salary : "+str(dropDisDF.count()))
dropDisDF.show()

Distinct count of department & salary : 8
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|      Michael|     Sales|  4600|
+-------------+----------+------+



In [16]:
df.sort("department","salary").show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|         Saif|     Sales|  4100|
|       Robert|     Sales|  4100|
|      Michael|     Sales|  4600|
+-------------+----------+------+



In [17]:
df.orderBy("department","salary").show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|         Saif|     Sales|  4100|
|       Robert|     Sales|  4100|
|      Michael|     Sales|  4600|
+-------------+----------+------+



In [18]:
df.sort(df.department.asc(),df.salary.asc()).show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|        Kumar| Marketing|  2000|
|         Jeff| Marketing|  3000|
|        James|     Sales|  3000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|         Saif|     Sales|  4100|
|      Michael|     Sales|  4600|
+-------------+----------+------+



In [19]:
df.createOrReplaceTempView("EMP")
spark.sql("select employee_name,department,salary from EMP ORDER BY department asc").show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        Maria|   Finance|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|        James|     Sales|  3000|
|       Robert|     Sales|  4100|
|      Michael|     Sales|  4600|
|        James|     Sales|  3000|
|         Saif|     Sales|  4100|
+-------------+----------+------+

